# Moller-Plesset

Las ecuaciones de Moller-Plesset surgen de aplicar la teoría de perturbaciones a Hartree-Fock. Para ello se toma como sistema conocido el Hamiltoniano de Hartree-Fock ($\mathcal{H}_0$) y se le aplica  una perturbación ($\mathcal{V}$) para convertirlo en el Hamiltoniano del sistema con correlación electrónica ($\mathcal{H}$).
\begin{equation}
\mathcal{H} = \mathcal{H}_0 + \mathcal{V}
\end{equation}

Para comenzar, importe las librerías numpy y psi4.

In [1]:
import numpy as np
import psi4

Defina una molécula de su interés. A continuación se proporciona la geometría de la molécula de hidrógeno, aunque puede usar cualquier otro sistema.
```
H 0.0000  0.0000 0.0000
H 0.0000  0.0000 0.7414 
```

In [2]:
mol = psi4.geometry("""
0 1
H 0.0000  0.0000 0.0000
H 0.0000  0.0000 0.7414 
units angstrom
symmetry C1
""")

Ya que la teoría de Moller-Plesset corrige el Hamiltoniano de Hartree-Fock, realice un cálculo de Hartree-Fock y recupere la función de onda. Recuerde que la energía resultante contiene tanto energía electrónica como nuclear. Seleccione la base que desee, por ejemplo, 6-31G

In [3]:
E_HF, wfn = psi4.energy('HF/6-31G',return_wfn=True)
print("E_HF =",E_HF)

E_HF = -1.1267350944766286


Obtenga la energía nuclear

In [4]:
E_nuc = mol.nuclear_repulsion_energy() 
print("E_nuc =",E_nuc)

E_nuc = 0.7137539933504182


Obtenga los coeficientes de los orbitales moleculares a partir de la función de onda

In [5]:
C = np.asarray(wfn.Ca())

Obtenga el número de funciones de base, de orbitales moleculares y de electrones

In [6]:
nbf = len(C)
nmo = wfn.nmo()
ne = wfn.nalpha() + wfn.nbeta()
print("nbf =",nbf," nmo =",nmo," ne =",ne)

nbf = 4  nmo = 4  ne = 2


Vamos a necesitar integrales de repulsión electónica
\begin{equation}
[\mu \nu | \sigma \lambda] = \int \int \frac{\mu(r_1) \nu(r_1) \sigma(r_2) \lambda(r_2)}{ |r_1 - r_2| } dr_1 dr_2
\end{equation}
donde $\mu$, $\nu$, $\sigma$ y $\lambda$ se refieren a orbitales atómicos.

Utilice las siguientes líneas para calcular las integrales $[\mu\nu|\sigma\lambda]$ y guardarlas en la variable I_AO
```
mints = psi4.core.MintsHelper(wfn.basisset())
I_AO = np.asarray(mints.ao_eri())
```

In [7]:
mints = psi4.core.MintsHelper(wfn.basisset())
I_AO = np.asarray(mints.ao_eri())

La teoría de Moller-Plesset requiere que estas integrales sean transformadas a orbital molecular. Recuerde que un orbital molecular es una combinación lineal de orbitales atómicos
\begin{equation}
p(r) = \sum_\mu C_{\mu p} \mu (r)
\end{equation}

En el siguiente recuadro declare una variable I_MO de dimensión ($N_{MO}$,$N_{MO}$,$N_{MO}$,$N_{MO}$), y lleve a cabo la transformación

\begin{equation}
[pq|rt] = \sum_{\mu\nu\sigma\lambda}^N C_{\mu p} C_{\nu q} C_{\sigma r} C_{\lambda s} [\mu \nu | \sigma \lambda]
\end{equation}

In [8]:
I_MO = np.einsum('mp,nq,sr,lt,mnsl->pqrt',C,C,C,C,I_AO,optimize=True)
#I_MO = np.zeros((nmo,nmo,nmo,nmo))
#for p in range(nmo):
#    for q in range(nmo):
#        for r in range(nmo):
#            for t in range(nmo):
#                for m in range(nbf):
#                    for n in range(nbf):
#                        for s in range(nbf):
#                            for l in range(nbf):
#                                I_MO[p][q][r][t] = I_MO[p][q][r][t] + C[m][p]*C[n][q]*C[s][r]*C[l][t]*I_AO[m][n][s][l]

También necesitamos las energías de los orbitales moleculares ($\varepsilon_a$), utilice las siguientes líneas para obtenerlas
```
epsilon = wfn.epsilon_a()
```

In [9]:
epsilon = np.array(wfn.epsilon_a())

Recordando teoría de perturbaciones, es posible realizar correcciones de n-orden a la energía $E_i^{(n)}$, tal que la energía electrónica total del estado basal ($i=0$) es
\begin{equation}
E = E_0^{(0)} + E_0^{(1)} + E_0^{(2)} + E_0^{(3)} + E_0^{(4)} ...
\end{equation}

dependiendo del n-orden hasta el que se haga la corrección sobre la energía al cálculo se le denomina MPn, siendo los más comunes son MP2, MP3 y MP4.

En Moller-Plesset estos términos son:
\begin{equation}
E_0^{(0)} = 2\sum_a^{N/2} {\varepsilon_a}
\end{equation}
\begin{equation}
E_0^{(1)} = -2 \sum_{ab}^{N/2} [aa|bb] - [ab|ba]
\end{equation}
donde los índices $a$, $b$ refieren a orbitales moleculares ocupados, y los términos $\varepsilon_a$, $\varepsilon_b$, a sus energías.

Calcule $E_0^{(0)}$ y $E_0^{(1)}$.

In [10]:
E_0 = 0
for a in range(int(ne/2)):
    E_0 = E_0 + 2*epsilon[a]
    
E_1 = 0
for a in range(int(ne/2)):
    for b in range(int(ne/2)):
        E_1 = E_1 - 2 * I_MO[a][a][b][b] + I_MO[a][b][b][a]

Calcule la energía total de MP1, recuerde sumar la energía nuclear, es decir

\begin{equation}
E_{MP1} = E_{nuc} + E_0^{(0)} + E_0^{(1)}
\end{equation}

In [11]:
E_MP1 = E_nuc + E_0 + E_1
print("E_MP1 =",E_MP1)

E_MP1 = -1.1267362218749755


Calcule la diferencia entre $E_{MP1}$ y la energía de Hartree-Fock que calculó al inicio, ¿Cuál es la energía de correlación? ¿Cómo se relaciona $E_{MP1}$ con $E_{HF}$?

In [12]:
# Respuesta

La energía de Hartree-Fock se calcula como
\begin{equation}
E_{HF} = E_{nuc} + 2\sum_a^{N/2} {\varepsilon_a} -2 \sum_{ab}^{N/2} [aa|bb] - [ab|ba]
\end{equation}

esta es exactamente la misma expresion que $E_{MP1}$. La primera corrección a la energía aparece en $E_{MP2}$. La corrección a segundo orden es
\begin{equation}
E_0^{(2)} = 2 \sum_{abrs}^{N/2} \frac{[ar|bs][ra|sb]}{\varepsilon_{a}+\varepsilon_{b}-\varepsilon_{r}-\varepsilon_{s}} - \sum_{abrs}^{N/2} \frac{[ar|bs][rb|sa]}{\varepsilon_{a}+\varepsilon_{b}-\varepsilon_{r}-\varepsilon_{s}}
\end{equation}

donde $r$, $s$ son los orbitales moleculares desocupados.

Calcule $E_0^{(2)}$, la energía de $MP2$ dada por
\begin{equation}
E_{MP2} = E_{nuc} + E_0^{(0)} + E_0^{(1)} + E_0^{(2)}
\end{equation}

y la energía de correlación
\begin{equation}
E_{corr} = E_{MP2} - E_{HF}
\end{equation}

In [13]:
E_2 = 0
for a in range(int(ne/2)):
    for b in range(int(ne/2)):
        for r in range(int(ne/2),nbf):
            for s in range(int(ne/2),nbf):
                E_2 = E_2 + 2*(I_MO[a][r][b][s]*I_MO[r][a][s][b])/(epsilon[a] + epsilon[b] - epsilon[r]- epsilon[s])
                E_2 = E_2 -   (I_MO[a][r][b][s]*I_MO[r][b][s][a])/(epsilon[a] + epsilon[b] - epsilon[r]- epsilon[s])
                
E_MP2 = E_nuc + E_0 + E_1 + E_2
print("E_MP2 =",E_MP2)

print("E_corr =",E_MP2-E_HF)

E_MP2 = -1.1441326668038203
E_corr = -0.017397572327191746


Generalizando, la energía total de MPn es
\begin{equation}
E_{MPn} = E_{nuc} + E_0^{(0)} + E_0^{(1)} + E_0^{(2)} + ... + E_0^{(n)}
\end{equation}

Al restarle la energía de Hartree-Fock se obtiene
\begin{equation}
E_{corr} = E_{MPn} - E_{HF} = E_0^{(2)} + ... + E_0^{(n)}
\end{equation}

Adapte la instrucción a MP2 junto con la base que usó para comprobar sus resultados.
```
E_MPn = psi4.energy('MPn/Base')
print("E_MPn =",E_MPn)
```

Ejemplo.
```
E_MP2 = psi4.energy('MP2/6-31G')
print("E_MP2 =",E_MP2)
```

In [14]:
E_MP2 = psi4.energy('MP2/6-31G')
print("E_MP2 =",E_MP2)

E_MP2 = -1.144114422972474
